# <center>Regularization in Neural Network</center>
<center>Shan-Hung Wu & DataLab<br/>Fall 2016</center>

In [3]:
from IPython.display import Image
from IPython.display import display

# inline plotting instead of popping out
%matplotlib inline

# load utility classes/functions that has been taught in previous labs
# e.g., plot_decision_regions()
import os, sys
module_path = os.path.abspath(os.path.join('.'))
sys.path.append(module_path)
from lib import *

Deep neural network with a large number of parameters is a powerful machine learning system, but overfitting is a serious issues in such networks. Deep neural network contains many non-linear hidden units and thus they can express very complicated relationships between their inputs and outputs. With limited training data, many of these relationships will be the result of sampling noise, so these relationships exist only on the training dataset, but not on the testing dataset. Large networks are slow, so it's hard for us to deal with the overfitting issues by training several networks with different architecture and combine their outputs.
In this lab, we are going to talk about regularization in neural network.
We will introduce some common regularization methods in deep neural network today, which is 
* Dropout
* Maxout
* Weight decay
* Adding noise

We will use the CIFAR-10 dataset today for our experiment. [CIFAR-10]() is a object recognition dataset of 10 class. The CIFAR-10 dataset consists of 60000 32x32 color images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 


The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class. 

Here are the classes in the dataset, as well as 10 random images from each:
<img src="fig-cifar-10.png" width="400">



# Loading and Preprocess the CIFAR-10 dataset

## Load data
set the path for storing the dataset on your machine

In [4]:
data_path = "data/CIFAR-10/"
data_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"

and some constants for processing the dataset

In [5]:
# the width and height of out image
img_size = 32
# number of channels: red, green and blue
img_channels = 3
# length of the image after we flatten the image into a 1-dim array
img_size_flat = img_size * img_size * img_channels
# number of classes
nb_classes = 10
# number of files in the training dataset
nb_files_train = 5
# number of images for each batch-file in the training-set.
images_per_file = 10000
# number of all the images in the training dataset
nb_images_train = nb_files_train * images_per_file

Then, we download and unzip the file. 

In [6]:
import os

# filename for saving the file downloaded from the internet.
filename = data_url.split('/')[-1]
file_path = os.path.join(data_path, filename)

# Check if the file already exists.
# If it exists then we assume it has also been extracted,
# otherwise we need to download and extract it now.
if not os.path.exists(file_path):
    # Check if the download directory exists, otherwise create it.
    if not os.path.exists(data_path):
        os.makedirs(data_path)

    # Download the file from the internet.
    file_path, _ = urllib.request.urlretrieve(url=data_url,
                                              filename=file_path,
                                              reporthook=_print_download_progress)

    print()
    print("Download finished. Extracting files.")

    if file_path.endswith(".zip"):
        # Unpack the zip-file.
        zipfile.ZipFile(file=file_path, mode="r").extractall(download_dir)
    elif file_path.endswith((".tar.gz", ".tgz")):
        # Unpack the tar-ball.
        tarfile.open(name=file_path, mode="r:gz").extractall(download_dir)

    print("Done.")
else:
    print("Data has already been downloaded and unpacked.")

Data has already been downloaded and unpacked.


Then, we load the classes name in the CIFAR-10 dataset from the metafile

In [7]:
import pickle
# Create full path for the file.
file_path = os.path.join(data_path, "cifar-10-batches-py/", "batches.meta")

print("Loading data: " + file_path)

with open(file_path, mode='rb') as file:
    # In Python 3.X it is important to set the encoding,
    # otherwise an exception is raised here.
    data = pickle.load(file, encoding='bytes')

raw = data[b'label_names']

# Convert from binary strings.
class_names = [x.decode('utf-8') for x in raw]
# class_names a list with the names. Example: names[3] is the name associated with class-number 3.
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [8]:
import numpy as np
from keras.utils import np_utils
def load_data(file_name):
    """
    Load a pickled data-file from the CIFAR-10 data-set
    and return the converted images (see above) and the class-number
    for each image.
    """

    # Create full path for the file.
    file_path = os.path.join(data_path, "cifar-10-batches-py/", file_name)

    print("Loading data: " + file_path)

    with open(file_path, mode='rb') as file:
        # In Python 3.X it is important to set the encoding,
        # otherwise an exception is raised here.
        data = pickle.load(file, encoding='bytes')

    # Get the raw images.
    raw_images = data[b'data']

    # Get the class-numbers for each image. Convert to numpy-array.
    cls = np.array(data[b'labels'])

    # Convert the images.
    """
    Convert images from the CIFAR-10 format and
    return a 4-dim array with shape: [image_number, channel, height, width]
    where the pixels are floats between 0.0 and 1.0.
    """

    # Convert the raw images from the data-files to floating-points.
    raw_float = np.array(raw_images, dtype=float) / 255.0

    # Reshape the array to a 4-dim array with shape: [image_number, channel, height, width] where the pixels are floats between 0.0 and 1.0.
    images = raw_float.reshape([-1, img_channels, img_size, img_size])

    return images, cls

def load_training_data():
    """
    Load all the training-data for the CIFAR-10 data-set.
    The data-set is split into 5 data-files which are merged here.
    Returns the images, class-numbers and one-hot encoded class-labels.
    """

    # Pre-allocate the arrays for the images and class-numbers for efficiency.
    images = np.zeros(shape=[nb_images_train, img_channels, img_size, img_size], dtype=float)
    cls = np.zeros(shape=[nb_images_train], dtype=int)

    # Begin-index for the current batch.
    begin = 0

    # For each data-file.
    for i in range(nb_files_train):
        # Load the images and class-numbers from the data-file.
        images_batch, cls_batch = load_data(file_name="data_batch_" + str(i + 1))

        # Number of images in this batch.
        num_images = len(images_batch)

        # End-index for the current batch.
        end = begin + num_images

        # Store the images into the array.
        images[begin:end, :] = images_batch

        # Store the class-numbers into the array.
        cls[begin:end] = cls_batch

        # The begin-index for the next batch is the current end-index.
        begin = end

    return images, cls, np_utils.to_categorical(cls, nb_classes)
X_train, cls_train, y_train = load_training_data()

Using Theano backend.
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5


Let's load the testing dataset using similar method

In [9]:
def load_test_data():
    """
    Load all the test-data for the CIFAR-10 data-set.

    Returns the images, class-numbers and one-hot encoded class-labels.
    """

    images, cls = load_data(file_name="test_batch")

    return images, cls, np_utils.to_categorical(cls, nb_classes)
X_test, cls_test, y_test = load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


Let's print out the size of the training and testing set to check if everything is loaded correctly.

In [10]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(X_train)))
print("- Test-set:\t\t{}".format(len(X_test)))

Size of:
- Training-set:		50000
- Test-set:		10000


Now our dataset is ready! Let's build a model on Keras to do the classification.
I use three convolutional layer followed by max-polling, one fully-connected(dense) layer and a softmax layer in my network. We will talk more about convolutional layer later.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, GaussianNoise, MaxoutDense
from keras.regularizers import l1, l2
from keras.optimizers import SGD
import matplotlib.pyplot as plt
import sys
import os
import time

# here are some settings for my network
batch_size = 32
nb_epoch = 50


model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=(img_channels, img_size, img_size)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# let's train the model using SGD + momentum
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

start_time = time.time()
%time his = model.fit(X_train, y_train, \
          batch_size=batch_size, \
          nb_epoch=nb_epoch, \
          validation_split=0.2, \
          shuffle=True) \

# evaluate our model
score = model.evaluate(X_test, y_test, verbose=1)
print('\nTest loss: %.3f' % score[0])
print('Test accuracy: %.3f' % score[1])


Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 235s - loss: 1.6083 - acc: 0.4170 - val_loss: 1.2983 - val_acc: 0.5361
Epoch 2/50
40000/40000 [==============================] - 226s - loss: 1.1706 - acc: 0.5850 - val_loss: 1.0824 - val_acc: 0.6162
Epoch 3/50
40000/40000 [==============================] - 231s - loss: 0.9372 - acc: 0.6690 - val_loss: 0.9483 - val_acc: 0.6703
Epoch 4/50
40000/40000 [==============================] - 227s - loss: 0.7499 - acc: 0.7375 - val_loss: 0.9094 - val_acc: 0.6900
Epoch 5/50
40000/40000 [==============================] - 230s - loss: 0.5893 - acc: 0.7928 - val_loss: 0.9481 - val_acc: 0.7014
Epoch 6/50
40000/40000 [==============================] - 226s - loss: 0.4460 - acc: 0.8442 - val_loss: 1.0390 - val_acc: 0.6900
Epoch 7/50
40000/40000 [==============================] - 228s - loss: 0.3469 - acc: 0.8787 - val_loss: 1.0985 - val_acc: 0.6984
Epoch 8/50
40000/40000 [=======================

In [ ]:
train_loss = his.history['loss']
val_loss = his.history['val_loss']

# visualize training history
plt.plot(range(1, len(train_loss)+1), train_loss, color='blue', label='Train loss')
plt.plot(range(1, len(val_loss)+1), val_loss, color='red', label='Val loss')
plt.legend(loc="upper right")
plt.xlabel('#Epoch')
plt.ylabel('Loss')
plt.savefig('./output/fig-nn-val-baseline.png', dpi=300)
plt.show()

Apparently we are having the issues of over fitting here. It's a chance for us to learn some regularizations technique here.

# Dropout
Dropout is designed by Geoffrey Hinton.
The key idea is to randomly drop some units from the neural network during training, so that the neuron have to function well on its own instead of relying on other neurons. (Just like when you know your teammates is not that reliable, you have to take more responsibility)
In a standard neural network, the derivative received by each parameter tells it how it should change so the final loss function is reduced, given what all other units are doing. Therefore, units may change in a way that they fix up the mistakes of the other units. This may lead to complex co-adaptations.
Applying dropout to a neural network amounts to sampling a “thinned” network from it. The thinned network consists of all the units that are not dropped out (Figure b). A neural net with $n$ units, can be seen as a collection of  possible thinned neural networks. These networks all share weights so that the total number of parameters is still $O(n^{2})$, or less. For each presentation of each training case, a new thinned network is sampled and trained.

The following figures illustrate the architecture:

<img src="fig-dropout.png" width="600">



The choice of which units to drop is random. In the simplest case, each unit is retained with a fixed probability $p$ independent of other units, where $p$ can be chosen using a validation set or can simply be set at 0.5, which seems to be close to optimal for a wide range of networks and tasks.
At test time, it is not feasible to explicitly average the predictions from exponentially many thinned models. A very simple approximate averaging method works well in practice. The idea is to use a single neural net at test time **without dropout**. If a unit is retained with probability $p$ during training, the outgoing weights of that unit are multiplied by $p$ at test time as shown in the figure below. This is to ensure that for any hidden unit the expected output (under the distribution used to drop units at training time) is the same as the actual output at test time.
<img src="fig-weight.png" width="600">

In [50]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=(img_channels, img_size, img_size)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
# Dropout layer with p = 0.25
model.add(Dropout(0.25))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
# add dropout
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
# add dropout
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))


sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

%time his = model.fit(X_train, y_train, \
          batch_size=batch_size, \
          nb_epoch=nb_epoch, \
          validation_split=0.2, \
          shuffle=True) \

# evaluate our model
score = model.evaluate(X_test, y_test, verbose=1)
print('\nTest loss: %.3f' % score[0])
print('Test accuracy: %.3f' % score[1])

Train on 40000 samples, validate on 10000 samples
Epoch 1/1
40000/40000 [==============================] - 345s - loss: 1.7790 - acc: 0.3420 - val_loss: 1.5292 - val_acc: 0.4622
CPU times: user 9min, sys: 26.2 s, total: 9min 27s
Wall time: 5min 56s
10000/10000 [==============================] - 33s    

Test loss: 1.533
Test accuracy: 0.459


In [ ]:
# plot_history(his)
train_loss = his.history['loss']
val_loss = his.history['val_loss']

# visualize training history
plt.plot(range(1, len(train_loss)+1), train_loss, color='blue', label='Train loss')
plt.plot(range(1, len(val_loss)+1), val_loss, color='red', label='Val loss')
plt.legend(loc="upper right")
plt.xlabel('#Epoch')
plt.ylabel('Loss')
plt.savefig('./output/fig-nn-val-baseline.png', dpi=300)
plt.show()

In [ ]:
We can see the dropout has a signifanct effect on the result!

# Maxout
Maxout is called maxout because its output is the max of a set of inputs. It wasa designed by Goodfellow on January 2013. 
You can simply create a dense layer with maxout by calling the MaxoutDense layer in Keras.
A MaxoutDense layer takes the element-wise maximum of nb_feature Dense(input_dim, output_dim) linear layers. This allows the layer to learn a convex, piecewise linear activation function over the inputs.
Given an input $v\subseteq\mathbb{R}^{d}$, a maxout hidden layer implements the function $h_{i}(x)=\underset{j\subseteq[1,k]}{max}z_{ij}$, where $z_{ij}=x^{T}W_{\text{···}ij}+b_{ij}$, and $W\subseteq\mathbb{R}^{d\times m\times k}$and $b\subseteq\mathbb{R}^{m\times k}$
<img src="fig-maxout.png" width="600">

# Weight Decay
Remember we talked about weight decay before? We can penalize large weights using penalties or constraints on their squared values (L2 penalty) or absolute values (L1 penalty).
We specify in l1/l2 regularities by passing a regularizer to the layer.
```
from keras.regularizers import l1, l2 
model.add(Dense(64, input_dim=64, W_regularizer=l2(0.01)))
```

## L1 weight cost

l1 regularizer will result in a lot of zeros in the weight.


## L2 weight cost
It makes a smoother model in which the output changes more slowly as the input changes.
If the network has two very similar inputs it prefers to put half the weight on each rather than all the weight on one.
We illustrate it one the following figure.
<img src="fig-smooth.png" width="400">
This can often improve generalization a lot because it helps to stop the network from fitting the sampling errorm and it makes a smoother model in which the output changes more slowly as the input changes.

# Adding Noise
We can add noise to input to prevent over-fitting. Previous work has shown that such training with noise is equivalent to a form of regularization in which an extra term is added to the error function.
in keras, you can do so by calling
```
keras.layers.noise.GaussianNoise(sigma)
```
This will apply to the input an additive zero-centered Gaussian noise with standard deviation sigma. This is useful to avoid overfitting